In [73]:
import tensorflow as tf
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
import pandas as pd
import numpy as np
import os
import torch
import gradio as gr

In [14]:
print(tf.__version__) 
print("Keras version:", tf.keras.__version__)

2.17.0
Keras version: 3.5.0


In [16]:
#Loading the data and Cleaning
df = pd.read_csv(r"train.csv")
df = np.array_split(df,1000)[0]
df

C:\Users\chris\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


,Conversation
0,The conversation between human and AI assistan...
1,The conversation between human and AI assistan...
2,The conversation between human and AI assistan...
3,The conversation between human and AI assistan...
4,The conversation between human and AI assistan...
...,...
102,The conversation between human and AI assistan...
103,The conversation between human and AI assistan...
104,The conversation between human and AI assistan...
105,The conversation between human and AI assistan...


In [18]:
df['Conversation1']=df['Conversation'].str.replace('\n', '')

In [20]:
df['Conversation1']=df['Conversation1'].str.replace('The conversation between human and AI assistant.[|Human|] ','')

In [22]:
df['Human']="User: "+df['Conversation1'].str.split("\[\|AI\|\]",expand=True)[0]
df['AI']="ChatBot: "+df['Conversation1'].str.split("\[\|AI\|\]",expand=True)[1]

<>:1: SyntaxWarning: invalid escape sequence '\['
<>:2: SyntaxWarning: invalid escape sequence '\['
<>:1: SyntaxWarning: invalid escape sequence '\['
<>:2: SyntaxWarning: invalid escape sequence '\['
C:\Users\chris\AppData\Local\Temp\ipykernel_25484\2224507441.py:1: SyntaxWarning: invalid escape sequence '\['
  df['Human']="User: "+df['Conversation1'].str.split("\[\|AI\|\]",expand=True)[0]
C:\Users\chris\AppData\Local\Temp\ipykernel_25484\2224507441.py:2: SyntaxWarning: invalid escape sequence '\['
  df['AI']="ChatBot: "+df['Conversation1'].str.split("\[\|AI\|\]",expand=True)[1]


In [24]:
df['Whole']= df['Conversation1'].str.replace('[|AI|]', '')

In [26]:
df_train=df['Whole']

In [28]:
df['Human'][0]

'User: Hi.My names Ahdieh.I m from a small city in Iran.My father had a heart attack on sunday as doctor said cpr condition.after 40 minuts he came back to life. now he isnot conscious.he is in ICU.his doctor said his conscious rate is 5.what do you think about his condition?If it is needed I can send you his cardiograf and blood test and the drugs they are using for him in ICU.Kindly tell me can we carry him to a better hospital in Tehran(It takes 4 hours to reach there)?'

In [34]:
df['Whole']

0      Hi.My names Ahdieh.I m from a small city in Ir...
1      Hello and thank you for your time and assistan...
2      Yes! I had double knee replacements on Feb. 20...
3      I m extremely itchy ALL OVER the place? I m ex...
4      Hi Doctor, I need your help, im really worried...
                             ...                        
102    I had a MRI done to look at a cyst that was fo...
103    hi m from pakistan. my age is 21 years n m fem...
104    I had a needle point white patch in the inner ...
105    HAVE BEEN EXPIERENCING WHAT THEY CALL BRAIN ZA...
106    when i play fast paced video games. after few ...
Name: Whole, Length: 107, dtype: object

In [6]:
# Load Pretrained Model and Tokeneizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [36]:
#Define Class Structure for DataLoader

class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        inputs = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )
        inputs['labels'] = inputs.input_ids.clone()
        return {key: val.squeeze(0) for key, val in inputs.items()}

In [42]:
#Creating dataset and dataloader variables

dataset = TextDataset(list(df['Whole']), tokenizer, max_length=512)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

In [46]:
# Add a padding token if not present (necessary for batching)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [51]:
#Defining Optimizer and Trining the model
optimizer = AdamW(model.parameters(), lr=5e-5)
model.train()
for epoch in range(5):
    for batch in dataloader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print(f'Epoch {epoch}, Loss: {loss.item()}')

C:\Users\chris\anaconda3\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 0, Loss: 9.867812156677246
Epoch 0, Loss: 7.91363000869751
Epoch 0, Loss: 8.202131271362305
Epoch 0, Loss: 8.185564041137695
Epoch 0, Loss: 7.954638481140137
Epoch 0, Loss: 8.737276077270508
Epoch 0, Loss: 7.946996212005615
Epoch 0, Loss: 7.985766410827637
Epoch 0, Loss: 8.048212051391602
Epoch 0, Loss: 7.8912248611450195
Epoch 0, Loss: 7.478363513946533
Epoch 0, Loss: 7.203813076019287
Epoch 0, Loss: 8.743584632873535
Epoch 0, Loss: 7.000365734100342
Epoch 0, Loss: 7.192269325256348
Epoch 0, Loss: 7.967451572418213
Epoch 0, Loss: 7.140986919403076
Epoch 0, Loss: 7.53182315826416
Epoch 0, Loss: 7.3150506019592285
Epoch 0, Loss: 7.713593482971191
Epoch 0, Loss: 7.625945568084717
Epoch 0, Loss: 7.357484340667725
Epoch 0, Loss: 7.088199138641357
Epoch 0, Loss: 6.396111488342285
Epoch 0, Loss: 6.804506301879883
Epoch 0, Loss: 7.018469333648682
Epoch 0, Loss: 7.126985549926758
Epoch 1, Loss: 5.821003437042236
Epoch 1, Loss: 6.431939125061035
Epoch 1, Loss: 6.367490291595459
Epoch 1, L

In [63]:
#Save Model

# Define a checkpoint directory
checkpoint_dir = './GPTModel'

# Ensure the directory exists
os.makedirs(checkpoint_dir, exist_ok=True)

# Save model checkpoint with unique filename (including epoch number)
model.save_pretrained(os.path.join(checkpoint_dir, f'model_epoch_{epoch}'))

# Save the optimizer state with unique filename
torch.save(optimizer.state_dict(), os.path.join(checkpoint_dir, f'optimizer_epoch_{epoch}.pt'))

# Save tokenizer
tokenizer.save_pretrained(os.path.join(checkpoint_dir, f'tokenizer_epoch_{epoch}'))

('./GPTModel\\tokenizer_epoch_8\\tokenizer_config.json',
 './GPTModel\\tokenizer_epoch_8\\special_tokens_map.json',
 './GPTModel\\tokenizer_epoch_8\\vocab.json',
 './GPTModel\\tokenizer_epoch_8\\merges.txt',
 './GPTModel\\tokenizer_epoch_8\\added_tokens.json')

In [64]:
#Load Model

# Get the latest checkpoint directory
latest_epoch = max([int(folder.split('_')[2]) for folder in os.listdir(checkpoint_dir) if 'model_epoch' in folder])

# Load the model from the latest checkpoint
model = GPT2LMHeadModel.from_pretrained(os.path.join(checkpoint_dir, f'model_epoch_{latest_epoch}'))

# Load optimizer state
optimizer = torch.optim.AdamW(model.parameters())  # Initialize optimizer
optimizer.load_state_dict(torch.load(os.path.join(checkpoint_dir, f'optimizer_epoch_{latest_epoch}.pt')))

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(os.path.join(checkpoint_dir, f'tokenizer_epoch_{latest_epoch}'))

C:\Users\chris\AppData\Local\Temp\ipykernel_25484\650055237.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  optimizer.load_state_dict(torch.load(os.path.join(checkpoint_

In [67]:
#Code to train back an fort
model.train()
for epoch in range(latest_epoch+1, latest_epoch+5):
    for batch in dataloader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print(f'Epoch {epoch}, Loss: {loss.item()}')

Epoch 9, Loss: 1.3381669521331787
Epoch 9, Loss: 1.1635215282440186
Epoch 9, Loss: 1.1753144264221191
Epoch 9, Loss: 1.1016569137573242
Epoch 9, Loss: 1.0951319932937622
Epoch 9, Loss: 1.2017995119094849
Epoch 9, Loss: 0.9554288983345032
Epoch 9, Loss: 1.2405176162719727
Epoch 9, Loss: 1.1678147315979004
Epoch 9, Loss: 1.3959702253341675
Epoch 9, Loss: 1.1414330005645752
Epoch 9, Loss: 1.3047692775726318
Epoch 9, Loss: 1.0674877166748047
Epoch 9, Loss: 1.018944501876831
Epoch 9, Loss: 1.1247917413711548
Epoch 9, Loss: 1.610595941543579
Epoch 9, Loss: 0.9228960871696472
Epoch 9, Loss: 1.1650652885437012
Epoch 9, Loss: 1.2311526536941528
Epoch 9, Loss: 1.387145757675171
Epoch 9, Loss: 1.1800094842910767
Epoch 9, Loss: 0.9026248455047607
Epoch 9, Loss: 1.2477115392684937
Epoch 9, Loss: 0.9140752553939819
Epoch 9, Loss: 1.11728835105896
Epoch 9, Loss: 1.1789889335632324
Epoch 9, Loss: 1.1690481901168823
Epoch 10, Loss: 1.177520990371704
Epoch 10, Loss: 1.0360409021377563
Epoch 10, Loss: 1.

In [79]:
#function to chat with the bot and Gradio 
def chat_with_bot(input_text):
    # Add task prefix
    model.eval()
    input_text = f"User: {input_text}"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    print(input_ids)
    output_ids = model.generate(input_ids, max_length=300)
    print(output_ids)
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return response

def predict_disease(input, history):
   
    return chat_with_bot(input)
    
interface = gr.ChatInterface(
    fn=predict_disease,
    title="DocRoboto",
    description="Predict diseases based on symptoms entered by the user.",
    theme="compact", 
)
interface.launch(share=True, inline=True)

C:\Users\chris\anaconda3\Lib\site-packages\gradio\blocks.py:1048: UserWarning: Cannot load compact. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/compact (Request ID: Root=1-675b6798-358cb6322c540f7e37bab9b3;1a88017d-4255-4ec9-8ab5-df4b1a756ab4)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")
C:\Users\chris\anaconda3\Lib\site-packages\gradio\components\chatbot.py:243: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7863

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
